In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import seaborn as sns
from sklearn.decomposition import PCA
# from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error, explained_variance_score, max_error

In [15]:
train_df=pd.read_csv(r'Train.csv')
test_df=pd.read_csv(r'Test.csv')

In [16]:
train_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,Time_Complete,Month,Day,Year,Time,Hour,Minutes,Seconds,UnitPrice,CustomerID,Country
0,6141,1583,144,3,06-05-2011 00:00,16:54:00,5,Fri,2011,4:54 PM,16,54,0,3.75,14056,35
1,6349,1300,3682,6,11-05-2011 00:00,07:35:00,5,Wed,2011,7:35 AM,7,35,0,1.95,13098,35
2,16783,2178,1939,4,20-11-2011 00:00,13:20:00,11,Sun,2011,1:20 PM,13,20,0,5.95,15044,35
3,16971,2115,2983,1,22-11-2011 00:00,12:07:00,11,Tue,2011,12:07 PM,12,7,0,0.83,15525,35
4,6080,1210,2886,12,06-05-2011 00:00,09:00:00,5,Fri,2011,9:00 AM,9,0,0,1.65,13952,35


In [17]:
train_df.loc[:,'InvoiceDate']=pd.to_datetime(train_df['InvoiceDate'])
test_df.loc[:,'InvoiceDate']=pd.to_datetime(test_df['InvoiceDate'])

In [25]:
train_df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [24]:
test_df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'CustomerID', 'Country'],
      dtype='object')

In [23]:
train_df= train_df.drop(['Time_Complete', 'Month', 'Day', 'Year', 'Time', 'Hour', 'Minutes', 'Seconds'],axis =1,inplace=False)
test_df =test_df.drop(['Time_Complete', 'Month', 'Day', 'Year', 'Time', 'Hour', 'Minute','Second'],axis = 1,inplace =False)

In [26]:

def split_data(train_data,test_data):
#     train_data['date'] = pd.to_datetime(train_data['DateTime'])
#     test_data['date'] = pd.to_datetime(test_data['DateTime'])

    train_data['month'] = train_data['InvoiceDate'].dt.month
    train_data['day'] = train_data['InvoiceDate'].dt.dayofweek
    train_data['year'] = train_data['InvoiceDate'].dt.year
    train_data['hour'] = train_data['InvoiceDate'].dt.hour
    train_data['day of week']=train_data['InvoiceDate'].dt.dayofweek
    
    test_data['month'] = test_data['InvoiceDate'].dt.month
    test_data['day'] = test_data['InvoiceDate'].dt.dayofweek
    test_data['year'] = test_data['InvoiceDate'].dt.year
    test_data['hour'] = test_data['InvoiceDate'].dt.hour
    test_data['day of week']=test_data['InvoiceDate'].dt.dayofweek
    return train_data,test_data

In [27]:
def model(train_x,train_y):
    params = {
        'nthread': 10,
         'max_depth': 10,
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression_l1',
        'metric': 'rmse', # this is abs(a-e)/max(1,a)

        'num_leaves': 64,
        'learning_rate': 0.2,
       'feature_fraction': 0.9,
       'bagging_fraction': 0.8,
        'bagging_freq': 15,
        'lambda_l1': 3.097758978478437,
        'lambda_l2': 2.9482537987198496,
        'verbose': 1,
        'min_child_weight': 6.996211413900573,
        'min_split_gain': 0.037310344962162616,
        }
    
    lgb_train = lgb.Dataset(train_x,train_y)
    model = lgb.train(params, lgb_train, 3000)
    y_test = model.predict(test_data_x)
    return y_test,model

In [28]:
train_data,test_data=split_data(train_df,test_df)

In [36]:
test_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country,month,day,year,hour,day of week
0,3139,1709,1079,3,2011-02-22 15:22:00,16161,35,2,1,2011,15,1
1,617,510,3457,1,2010-08-12 12:46:00,17341,35,8,3,2010,12,3
2,14653,604,694,36,2011-10-25 13:53:00,15158,35,10,1,2011,13,1
3,8634,1478,3473,2,2011-06-27 12:38:00,16033,35,6,0,2011,12,0
4,15546,3216,871,1,2011-06-11 16:14:00,15351,35,6,5,2011,16,5


In [50]:
test_data['InvoiceNo'].unique

<bound method Series.unique of 0          3139
1           617
2         14653
3          8634
4         15546
          ...  
122044     7813
122045     8694
122046      915
122047     8156
122048    12020
Name: InvoiceNo, Length: 122049, dtype: int64>

In [33]:
train_data_x = train_data.drop(['UnitPrice','InvoiceDate'],1)

In [37]:
test_data_x = test_data.drop(['InvoiceDate'],1)

In [29]:
train_data_y=train_data['UnitPrice']

In [34]:
train_data_x.head()

,InvoiceNo,StockCode,Description,Quantity,CustomerID,Country,month,day,year,hour,day of week
0,6141,1583,144,3,14056,35,6,6,2011,0,6
1,6349,1300,3682,6,13098,35,11,5,2011,0,5
2,16783,2178,1939,4,15044,35,11,6,2011,0,6
3,16971,2115,2983,1,15525,35,11,1,2011,0,1
4,6080,1210,2886,12,13952,35,6,6,2011,0,6


In [38]:
y_test, model = model(train_data_x,train_data_y)

In [41]:
y_test.shape

(122049,)

In [42]:
import pandas as pd
y_test=pd.DataFrame(y_test)

In [44]:
y_test.columns= ['UnitPrice']

In [46]:
y_test.to_csv('submission_lgbm1.csv',index=False)

### KNN

In [49]:
# Using K_Nearest neighbour regressor
# running for different K values to know which yields the max accuracy.
from sklearn.neighbors import KNeighborsRegressor
score = []
for k in range(1,20):    
    clf = KNeighborsRegressor(n_neighbors = k,  weights = 'distance', p=1)
    clf.fit(train_data_x, train_data_y)
    score.append(clf.score(train_data_x,test_data_x))

ValueError: Found input variables with inconsistent numbers of samples: [122049, 284780]

In [ ]:
k_max = score.index(max(score))+1
print( "At K = {}, Max Accuracy = {}".format(k_max, max(score)*100))